In [ ]:
!pip install mysql-connector-python
#above for first set of examples
!pip install python-dotenv
!pip install sqlalchemy
!pip install pymysql
#pymysql for the sqlalchemy example at end

import mysql.connector

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import mysql.connector

#bad way first
try:
    cnx = mysql.connector.connect(user='root', password='p4ssword',host='127.0.0.1',database='feb9')
    if cnx.is_connected():
        cursor = cnx.cursor()

    cursor.execute("DROP TABLE IF EXISTS sport;")
    cursor.execute("""CREATE TABLE sport (
                sportID int,
                sport varchar(255)); """)
    cursor.execute("""INSERT INTO sport VALUES
                    (1,'Basketball'),
                    (2,'Football'),
                    (3,'Hockey');
    """)

    cursor.execute("SELECT * FROM sport;")
    for row in cursor:
        print(row)

    cursor.close()
    cnx.close()
        
except Exception as e:
    print("Error while connecting to MySQL:", e)









(1, 'Basketball')
(2, 'Football')
(3, 'Hockey')


In [ ]:
#better way
from dotenv import load_dotenv
import os

load_dotenv("my_credentials.env")
#ADD my_credentials.env to .gitignore, or just add all .env files to .gitignore

db_user=os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host=os.getenv("DB_HOST")
db_name=os.getenv("DB_NAME")


try:
    cnx = mysql.connector.connect(user=db_user, password=db_password,host=db_host,database=db_name)
    if cnx.is_connected():
        cursor = cnx.cursor()

    cursor.execute("DROP TABLE IF EXISTS sport;")
    cursor.execute("""CREATE TABLE sport (
                sportID int,
                sport varchar(255)); """)
    cursor.execute("""INSERT INTO sport VALUES
                    (1,'Basketball'),
                    (2,'Football'),
                    (3,'Hockey');
    """)

    cursor.execute("SELECT * FROM sport;")
    for row in cursor:
        print(row)

    # cursor.close()
    # cnx.close()
        
except Exception as e:
    print("Error while connecting to MySQL:", e)





(1, 'Basketball')
(2, 'Football')
(3, 'Hockey')


In [29]:
#bad way to take user input, vulnerbale to Injection Attack

cursor.execute("DROP TABLE IF EXISTS players;")
cursor.execute("""CREATE TABLE players (
               playerID int,
               firstname varchar(255),
               lastname varchar(255),
               sport varchar(255)); """)
cursor.execute("""INSERT INTO players VALUES
                (1,'Wayne','Gretsky','hockey'),
                (2,'Michael','Jordan','basketball');
""")

cnx.commit()

while True:
    check = input("do you want to add a player to the database? (y,n)")
    if check != "y":
        break

    playerId = int(input("Enter player ID:"))
    firstname = input("Enter player first name:")
    lastname = input ("Enter player last name:")
    sport = input ("Enter player's sport")

    stmt = "INSERT INTO players VALUES ("+str(playerId)+", '"+firstname+"', '"+lastname +"', '"+sport +"');"
    cursor.execute(stmt)

    cnx.commit()

# cursor.execute("SELECT * FROM players;")
# for row in cursor:
#     print (row)




#allow user to to add a player
# could do something like this: basketball'); DROP TABLE players; --



In [30]:
#GOOD way to take user input, NOT vulnerbale to Injection Attack
#better way
from dotenv import load_dotenv
import os

load_dotenv("my_credentials.env")
#ADD my_credentials.env to .gitignore, or just add all .env files to .gitignore

db_user=os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host=os.getenv("DB_HOST")
db_name=os.getenv("DB_NAME")


try:
    cnx = mysql.connector.connect(user=db_user, password=db_password,host=db_host,database=db_name)
    if cnx.is_connected():
        cursor = cnx.cursor()

    cursor.execute("DROP TABLE IF EXISTS players;")
    cursor.execute("""CREATE TABLE players (
                playerID int,
                firstname varchar(255),
                lastname varchar(255),
                sport varchar(255)); """)
    cursor.execute("""INSERT INTO players VALUES
                    (1,'Wayne','Gretsky','hockey'),
                    (2,'Michael','Jordan','basketball');
    """)

    cnx.commit()

    while True:
        check = input("do you want to add a player to the database? (y,n)")
        if check != "y":
            break

        playerId = int(input("Enter player ID:"))
        firstname = input("Enter player first name:")
        lastname = input ("Enter player last name:")
        sport = input ("Enter player's sport")

        stmt = """INSERT INTO players VALUES (%s,%s,%s,%s)
        """
        values = (playerId,firstname,lastname,sport)
        cursor.execute(stmt,values)

        cnx.commit()

    cursor.execute("SELECT * FROM players;")
    for row in cursor:
        print (row)

    cursor.close()
    cnx.close()

    # cursor.close()
    # cnx.close()
        
except Exception as e:
    print("Error while connecting to MySQL:", e)









#allow user to to add a player
# could do something like this: basketball'); DROP TABLE players; --
# but now above just inserts above as a string rather than an injection attack deleting table
# 




(1, 'Wayne', 'Gretsky', 'hockey')
(2, 'Michael', 'Jordan', 'basketball')
(3, 'lindsay', 'Vonn', 'skiing')


In [5]:
# or instead we could use an ORM like sql alchemy
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from pymysql import *



#GOOD way to take user input, NOT vulnerbale to Injection Attack

#better way
from dotenv import load_dotenv
import os

load_dotenv("my_credentials.env")
#ADD my_credentials.env to .gitignore, or just add all .env files to .gitignore

db_user=os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host=os.getenv("DB_HOST")
db_name=os.getenv("DB_NAME")

Base= declarative_base()

#Define ORM model (this replaces the previous CREATE TABLE logic)
class Player(Base):
    __tablename__ = "players"
    playerID = Column(Integer,primary_key=True)
    firstname = Column(String(255),nullable=False)
    lastname = Column(String(255), nullable=False)
    sport = Column(String(255),nullable=False)


try:
    engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
                           echo=False) # set True for generated SQL
    
    #drop players first
    Base.metadata.drop_all(
    engine,
    tables=[Player.__table__],
    checkfirst=True
    )
    
    #create tables if they don't exist
    Base.metadata.create_all(engine)

    #Create session (replaced cursor)
    Session = sessionmaker(bind=engine)
    session= Session()

    print("Connected using SQLAlchemy")

    #Insert example
    serena = Player(firstname="Serena",lastname="Williams",sport="Tennis")
    session.add(serena)
    session.commit()

    while True:
        check = input("do you want to add a player to the database? (y,n)")
        if check != "y":
            break

        playerID = int(input("Enter player ID:"))
        firstname = input("Enter player first name:")
        lastname = input ("Enter player last name:")
        sport = input ("Enter player's sport")

        new_player = Player(
            playerID= playerID,
            firstname = firstname,
            lastname = lastname,
            sport = sport
        )

        session.add(new_player)
        session.commit()
    
    
    session.close()
    engine.dispose()
       
except SQLAlchemyError as e:
    print("Database error:", e)

    
    




#allow user to to add a player
# could do something like this: basketball'); DROP TABLE players; --
# but now above just inserts above as a string rather than an injection attack deleting table
# 




Connected using SQLAlchemy
